In [26]:
import time

train_data = pd.read_csv('raw_data/train.csv')
test_data = pd.read_csv('raw_data/test.csv')
raw_data=(train_data,test_data)
for df in raw_data:
    df['date_str']=np.core.defchararray.replace(df.Date.values.astype('U'),'-','')
    df['date_int']=df.date_str.astype('int')

result = test_data.copy()
result['Id']=test_data.Store.astype('U')+'_'+test_data.Dept.astype('U')+'_'+test_data.Date.astype('U')
result['Weekly_Sales']=0

t = time.time()
store=-1
dept=-1
dept_train=None
for i in range(len(result)):
    
    s=result.loc[i,'Store']
    d=result.loc[i,'Dept']
    date=np.datetime64(result.loc[i,'Date'])
    
    if s != store or d != dept:
        store = s
        dept =d
        dept_train=train_data[np.logical_and(train_data.Store==store,train_data.Dept==dept)]
    
    history_date = date+np.timedelta64(-52,'W')
    history_sale = dept_train[dept_train.date_int==int(str(history_date).replace('-',''))]
    if len(history_sale) == 1:
        result.loc[i,'Weekly_Sales'] = history_sale.iloc[0]['Weekly_Sales']
    
print('total time %ds'%int(time.time()-t))
result.head()


total time 169s


,Store,Dept,Date,IsHoliday,date_str,date_int,Id,Weekly_Sales
0,1,1,2012-11-02,False,20121102,20121102,1_1_2012-11-02,39886.06
1,1,1,2012-11-09,False,20121109,20121109,1_1_2012-11-09,18689.54
2,1,1,2012-11-16,False,20121116,20121116,1_1_2012-11-16,19050.66
3,1,1,2012-11-23,True,20121123,20121123,1_1_2012-11-23,20911.25
4,1,1,2012-11-30,False,20121130,20121130,1_1_2012-11-30,25293.49


In [28]:
len(result[result.Weekly_Sales==0])

2050

In [29]:
result[['Id','Weekly_Sales']].to_csv('output/naive/result.csv',header=True,index=False)

score: 2943
rank: 113/692

In [ ]:
def score(predict_data,validate_data):
    validate_data['Weight'] = np.array([5 if holiday else 1 for holiday in validate_data['IsHoliday']])
    return np.sum(np.abs((predict_data.Weekly_Sales-validate_data.Weekly_Sales)*validate_data.Weight)) / np.sum(validate_data.Weight)

class NaiveModel:
    def __init__(self):
        pass
    def fit(self, train_data):
        self.train_data=train_data
        
    def predict(self, test_data):
        predict_data = test_data.copy()
        predict_data['Weekly_Sales']=0
        
        for date in predict_data.index:
            history_date = date+np.timedelta64(-52,'W')
            history_sale = self.train_data[self.train_data.index==history_date]
            if len(history_sale) == 1:
                predict_data.loc[date,'Weekly_Sales'] = history_sale.loc[history_date, 'Weekly_Sales']
                
        return predict_data
    
naive_m = NaiveModel()
naive_m.fit(dept_train)
dept_predict=naive_m.predict(dept_test)
score(dept_predict,dept_test)
